In [6]:
import requests, bs4
import pandas as pd
# Вводим тикер актива и даты начала и конца.
TICKER = 'SBER'
DATE_FROM = '2013-01-01'
DATE_TILL = '2018-08-15'
start = 0
hlocv = []
while True:
    # Собираем информацию об инструменте, для запроса рыночных данных.
    ticker_info = requests.get('http://iss.moex.com/iss/securities.xml?q=%s' % (TICKER)).text
    ticker_info = bs4.BeautifulSoup(ticker_info, 'html.parser') 
    ticker_info = ticker_info.find('rows').find('row', {'secid': '%s' % (TICKER)})
    engine = ticker_info.get('group').split('_')[0]
    market = ticker_info.get('group').split('_')[1]
    board = ticker_info.get('primary_boardid')
    # Запрашиваем маркет дату.
    items = requests.get('http://iss.moex.com/iss/history/engines/%s/markets/%s/boards/%s/securities/%s/candles.xml?from=%s&till=%s&interval=0&start=%d' % (engine, market, board, TICKER, DATE_FROM, DATE_TILL, start)).text
    items = bs4.BeautifulSoup(items, 'html.parser') 
    items = items.find('rows').find_all('row')
    # Составляем словарь из параметров торгового дня и формируем один большой список из всех дней диапазона дат.
    for i in items:
        date = i.get('tradedate')
        open_p = i.get('open')
        high = i.get('high')
        low = i.get('low')
        close = i.get('close')
        volume = i.get('volume')
        hlocv.append({'date': date, 'open': open_p, 'high': high, 'low': low, 'close': close, 'volume': volume })
    # Проверяем является ли дата последнего торгового дня в выдаче конечной датой нашего интервала.    
    if items[-1].get('tradedate') != DATE_TILL:
        # Если нет, то загружаем еще
        start +=len(items)
    else:
        break
# формируем датафрейм
df = pd.DataFrame(hlocv, columns = ['date','open', 'high', 'low', 'close', 'volume'])
df

,date,open,high,low,close,volume
0,2013-03-25,96,101.14,96,98.79,593680
1,2013-03-26,98.58,99.31,97.08,97.2,1283550
2,2013-03-27,97.9,98,95.39,96.75,1261950
3,2013-03-28,96.38,98.66,95.72,98.59,1971410
4,2013-03-29,98.6,99.09,98.32,98.76,782000
5,2013-04-01,98.37,99.43,97.55,98.07,2279650
6,2013-04-02,98.12,99.45,98.04,98.93,3555020
7,2013-04-03,98.9,99.71,98.69,99.59,3757240
8,2013-04-04,98.86,101.13,98.5,99.91,5172930
9,2013-04-05,99.88,100.84,98.4,99.38,5229210
